# Public EV Charger Placement Optimization Project

## EVCS-OPTIM
Electric Vehicle Charging Station - Optimization Placement Tool via Intersection Modeling

In [ ]:
import pandas as pd
import numpy as np
import requests
import geopandas as gpd
import matplotlib.pyplot as plt


import folium
from folium.plugins import HeatMap
import branca

import osmnx as ox
from geopy.distance import geodesic
from shapely.geometry import Point, Polygon, mapping, MultiPolygon

**Brainstorm**

Top Charging Locations by Type:
- Shopping Centers
- Other commercial destinations
- Offices (Semi-Private)
- Parks
- Libraries
- Schools
- Other public spaces
- Hospitals/Clinics
- Public Transit Stations
- Rest/Refuel Stations
- High Density Residential/Mixed
- Community Centers / Churches
- Stadiums

**Consumer Costs**


## Data Gathering

### Charger Data

Let's start our data gathering process by collecting information on public EV chargers in California through AFDC database

In [ ]:
# Charger Data Query
url = "https://developer.nrel.gov/api/alt-fuel-stations/v1.json"
params = {
    'format': 'json',  # Output response format
    'api_key': 'Tbqhfv28h6gIIxbaMTByUNg4ByP2vf1E1A3XYgGa',  # Your developer API key
    'status': 'all',  # Return stations that match the given status
    # 'access': 'public',  # Return stations with the given access type
    'fuel_type': 'ELEC', # Return stations that supply any of the given fuel types
    'state': 'CA',
    'country': 'US',
    
}

response = requests.get(url, params=params)
if response.status_code == 200:

    data = response.json()

### SDG&E Territory Zip Codes

We will need to use SDG&E territory zip codes to filter much of our data by the relevant geographical area

In [ ]:
sdge_zips_data = pd.read_excel("data/SDGE Service Territory Zip Code List Q2-2021.xlsx")
sdge_zips = list(sdge_zips_data['ZIP_CODE'])

### Zoning Data

Zoning regulations and land-use can be a useful determinant of where EV chargers might be located

In [ ]:
zoning_data = gpd.read_file("shapefiles/zoning_datasd.geojson")

zoning_data = zoning_data.to_crs(epsg=4326)  # Convert to WGS84 (EPSG:4326)

# Convert zone codes to types
zoning_categories = {
    'Commercial': ['CC', 'CN', 'CV', 'CP', 'CR', 'CCPD'],
    'Office': ['CO'],
    'Residential High': ['RH', 'RM-3', 'RM-4'],
    'Residential Medium': ['RM-2', 'RM-1'],
    'Residential Low': ['RS', 'RL'],
    'Residential Mixed': ['RMX'],
    'Industrial': ['IP', 'IL', 'IH', 'IS', 'IBT'],
    'Mixed Use': ['MU', 'EMX'],
    'Agricultural': ['AG', 'AR'],
    'Open Space': ['OS'],
    'Planned': ['BLPD', 'MBPD', 'GQPD', 'MPD', 'CUPD', 'LJPD', 'LJSPD'],
    'Transit': ['OTOP', 'OTRM', 'OTCC'],
    'Other': ['UNZONED'],
}

def map_zoning_category(zone_code):
    if isinstance(zone_code, str):  # Check if zone_code is a string
        for category, prefixes in zoning_categories.items():
            if any(zone_code.startswith(prefix) for prefix in prefixes):
                return category
    return 'Other'  # Return 'Other' for NaN or non-string values

zoning_data.fillna({'zone_name':'Unknown'}, inplace=True)
zoning_data['zone_type'] = zoning_data['zone_name'].apply(map_zoning_category)

### DMV Registration Data

Gather information on the local registration information of vehicles according to the DMV

In [ ]:
dmv_data = pd.read_csv("shapefiles/vehicle-fuel-type-count-by-zip-code-2023.csv")


### Installation Costs

- 2015 EVCS/EVSE Installation Costs: https://afdc.energy.gov/files/u/publication/evse_cost_report_2015.pdf
  - Average public AC level 2 EVCS installation cost in San Diego is ~4000 (33% higher than average)
    
  - Average public DCFC EVCS installation cost ~24000 (33% figure would entail ~32000 cost in San Diego)



## Pre-Processing

### Zip Codes

In [ ]:
pop_gdf = gpd.read_file("data/population.geojson", driver="GeoJSON")
income_gdf = gpd.read_file("data/income.geojson", driver="GeoJSON")
# pop_gdf.head()

In [ ]:
pop_gdf

### Parking Lots

In [ ]:
zip_shape = gpd.read_file("shapefiles/sdge_zcta.shp")
# zip_shape

In [ ]:
zip_shape

In [ ]:
import osmnx as ox
import geopandas as gpd
import folium

# Define the area of interest
place_name = "San Diego, California, USA"

# Query OSM for parking lots
parking_lots = ox.geometries_from_place(place_name, tags={"amenity": "parking"})

# Filter for public parking lots
public_parking = parking_lots[
    (parking_lots.get("parking") == "public") |  # Explicitly public parking
    (parking_lots.get("access").isin([None, "yes", "permissive"]))  # Exclude private/restricted lots
]

# Convert to a GeoDataFrame
public_parking_gdf = gpd.GeoDataFrame(public_parking, geometry="geometry", crs="EPSG:4326")
parking_cols = ['osmid','geometry','amenity','access','fee','parking','capacity']
public_parking_gdf = public_parking_gdf.reset_index()[parking_cols]

# Create a folium map centered on San Diego
m = folium.Map(location=[32.7157, -117.1611], zoom_start=12)

# Add public parking lots to the map
for _, row in public_parking_gdf.iterrows():
    if row.geometry.geom_type == "Point":
        folium.CircleMarker(
            location=[row.geometry.y, row.geometry.x],
            radius=5,
            color="blue",
            fill=True,
            fill_color="blue",
            fill_opacity=0.6,
            popup="Public Parking Lot"
        ).add_to(m)
    elif row.geometry.geom_type in ["Polygon", "MultiPolygon"]:
        folium.GeoJson(
            row.geometry,
            style_function=lambda x: {"color": "blue", "fillColor": "blue", "weight": 1, "fillOpacity": 0.4},
            tooltip="Non-Private Parking Lot"
        ).add_to(m)
print(public_parking_gdf.shape)
print(public_parking_gdf.columns)
m

In [ ]:
public_parking_gdf.head(5)

In [ ]:
# intersections_gdf = intersections_gdf.set_crs("EPSG:4326", allow_override=True)
zoning_data = zoning_data.set_crs("EPSG:4326", allow_override=True)

# Perform the spatial join to map intersections to zoning data
lots = gpd.sjoin(public_parking_gdf, zoning_data, how="left", predicate="within")

# Drop irrelevant columns
lots = lots.drop(columns=['index_right','imp_date','ordnum','objectid'])

# Remove intersections with null zoning
lots.loc[pd.isna(lots['zone_type']), 'zone_type'] = "Multiple"

print(lots.shape)
lots.head()

In [ ]:
lots.value_counts('zone_type')

In [ ]:
import folium
import branca.colormap as cm

# Create a folium map centered on San Diego
m2 = folium.Map(location=[32.7157, -117.1611], zoom_start=12)

# Define color mapping for zoning types
color_mapping = {
    'Commercial': 'lightblue',
    'Office': 'lightgreen',
    'Residential High': 'lightcoral',
    'Residential Medium': 'khaki',
    'Residential Low': 'lightyellow',
    'Residential Mixed': 'lightpink',
    'Industrial': 'lightgray',
    'Mixed Use': 'lightseagreen',
    'Agricultural': 'lightgoldenrodyellow',
    'Open Space': 'lightblue',
    'Planned': 'lightcyan',
    'Transit': 'lavender',
    'Other': 'gray',
}

# Function to style zoning polygons
def zoning_style(feature):
    zone_type = feature['properties']['zone_type']
    color = color_mapping.get(zone_type, 'gray')  # Default to gray if not found
    return {
        'fillColor': color, 
        'color': 'black', 
        'weight': 1,
        'fillOpacity': 0.3,
    }
    
# Add zoning overlay to the map
folium.GeoJson(
    zoning_data,  # GeoJSON file with zoning polygons
    style_function=zoning_style
).add_to(m2)

# Add a legend for zoning categories
opacity = 0.8
legend_html = f"""
<div style="position: fixed; 
             top: 10px; left: 10px; 
             width: 200px; height: auto; 
             background-color: rgba(255, 255, 255, {opacity}); 
             border:2px solid grey; 
             z-index: 9999; 
             font-size:14px;">
    <b>Zoning Categories</b><br>
    <div style="line-height: 1.5;">
"""
for category, color in color_mapping.items():
    legend_html += f'<div><i style="background:{color}; width: 20px; height: 20px; display: inline-block;"></i> {category}</div>'
legend_html += "</div></div>"
m2.get_root().html.add_child(folium.Element(legend_html))

# Add parking lots with zoning type tooltips
for _, row in lots.iterrows():
    zone_type = row['zone_type'] if 'zone_type' in row else "Unknown"
    
    if row.geometry.geom_type == "Point":
        folium.CircleMarker(
            location=[row.geometry.y, row.geometry.x],
            radius=5,
            color="green",
            fill=True,
            fill_color="green",
            fill_opacity=0.7,
            popup=f"Zone Type: {zone_type}",
            tooltip=f"Zone: {zone_type}"
        ).add_to(m2)
    
    elif row.geometry.geom_type in ["Polygon", "MultiPolygon"]:
        folium.GeoJson(
            row.geometry,
            style_function=lambda x: {"color": "green", "fillColor": "green", "weight": 1, "fillOpacity": 0.4},
            tooltip=f"Zone: {zone_type}"
        ).add_to(m2)


# Save map as an HTML file (optional)
m2.save("parking_lots_with_zoning.html")

# Display the map
print("Map of Public Parking Lots Overlaid by City Zoning")
m2

### AFDC Charging Stations

In [ ]:
# CS - Charging Stations
cs = pd.DataFrame(data['fuel_stations'])
cs = cs[cs['zip'] != 'CA']
cs['zip'] = cs['zip'].astype(int)
cs = cs[cs['zip'].isin(sdge_zips)]
cs = cs[cs['access_code']=="public"]
cs = cs[[
    'id', 'access_code','facility_type','latitude','longitude','zip','ev_connector_types',
    'ev_dc_fast_num','ev_level1_evse_num','ev_level2_evse_num','ev_network'
]]
cs = cs.reset_index(drop=True)

print(cs.shape)
# cs.columns
cs.head()

In [ ]:
%%time
# Create a geometry column for EV chargers
geometry = [Point(xy) for xy in zip(cs['longitude'], cs['latitude'])]
cs_gdf = gpd.GeoDataFrame(cs, geometry=geometry)
cs_gdf.set_crs(epsg=4326, inplace=True)

# Create a one-hot encoding of the 'ev_connector_types' column
# Step 1: Explode the 'ev_connector_types' column to create individual rows
cs_gdf_exploded = cs_gdf.explode('ev_connector_types', ignore_index=True)

# Step 2: One-hot encode the exploded 'ev_connector_types' column
one_hot_encoded = pd.get_dummies(cs_gdf_exploded['ev_connector_types'], prefix='connector')
connector_cols = list(one_hot_encoded.columns)

# Step 3: Combine the one-hot encoded DataFrame with the exploded DataFrame
cs_gdf_exploded = pd.concat([cs_gdf_exploded, one_hot_encoded], axis=1)
cs_gdf_exploded = cs_gdf_exploded.groupby('id')[connector_cols].sum().reset_index()

# Step 4: Aggregate back to the original DataFrame, summing up the one-hot columns
cs_gdf = cs_gdf.merge(cs_gdf_exploded, how='left', on='id')
cs_gdf = cs_gdf.drop(columns=['ev_connector_types'])

\

# Check the resulting DataFrame
print(cs_gdf.shape)
cs_gdf.head()

In [ ]:
lots_cs = gpd.read_file("lots_cs.geojson")
# lots_cs = lots_cs.iloc[:,1:]
lots_cs.head(2)

In [ ]:
rl = gpd.read_file("data/roads_datasd.geojson")
print(rl.shape)
rl.head(2)

In [ ]:
tc = pd.read_csv("data/traffic_counts_datasd.csv")
# print(tc.shape)
tc['date_count'] = pd.to_datetime(tc['date_count'])
tc_sorted = tc.sort_values(by=['street_name', 'date_count'], ascending=[True, False])
tc_recent = tc_sorted.drop_duplicates(subset='street_name')
print(tc_recent.shape)
tc_recent.head(2)

In [ ]:
from shapely.geometry import MultiLineString
rl = rl.to_crs(epsg=4326)

# Merge based on street names and ensure the result is a GeoDataFrame
tc_rel = tc_recent[['id','street_name','total_count']]
rl_rel = rl[['rd20full','geometry']]
merged = tc_rel.merge(rl_rel, left_on='street_name', right_on='rd20full', how='inner')

# Convert the merged DataFrame to a GeoDataFrame
road_traffic = gpd.GeoDataFrame(merged, geometry='geometry')

# Group by 'id', aggregate geometries into MultiLineString, and calculate the average 'total_count'
road_traffic_grouped = road_traffic.groupby('street_name').agg({
    'geometry': lambda x: MultiLineString(x.tolist()),
    'total_count': 'mean'
}).reset_index()

# Convert the grouped DataFrame to a GeoDataFrame and set the CRS
road_traffic_grouped = gpd.GeoDataFrame(road_traffic_grouped, geometry='geometry')
road_traffic_grouped.crs = 'EPSG:4326'

# Print the final road_traffic_grouped GeoDataFrame and count of NaN in 'total_count'
print(road_traffic_grouped['total_count'].isna().sum())
road_traffic_grouped.head(2)

In [ ]:
lots_cs = gpd.read_file("lots_cs.geojson")

def get_nearest_charger_zip(lot_row, cs_gdf):
    if isinstance(lot_row.geometry, Point):
        lot_coords = (lot_row.geometry.y, lot_row.geometry.x)
    else:
        lot_coords = (lot_row.geometry.centroid.y, lot_row.geometry.centroid.x)
    
    distances = cs_gdf.apply(
        lambda row: geodesic(lot_coords, (row.latitude, row.longitude)).meters, 
        axis=1
    )
    
    nearest_index = distances.idxmin()
    return cs_gdf.loc[nearest_index, 'zip']

lots_cs['zip'] = lots_cs.apply(lambda row: get_nearest_charger_zip(row, cs_gdf), axis=1)

cs_lots_traffic = lots_cs.copy()

In [ ]:
%%time
# Assuming you have your dataframes ready: road_traffic and cs_lots
# Ensure they both have a consistent coordinate reference system (CRS)
road_traffic_grouped = road_traffic_grouped.to_crs('EPSG:4326')
lots_cs = lots_cs.to_crs('EPSG:4326')

cs_lots_traffic = lots_cs.copy()

# Re-project to a projected CRS for accurate buffer operation (e.g., EPSG:3857)
buffered_cs_lots = lots_cs.to_crs('EPSG:3857')
road_traffic_projected = road_traffic_grouped.to_crs('EPSG:3857')

# Buffer the parking lots to consider "adjacent" roads within a given distance (e.g., 5 meters)
buffered_cs_lots['geometry'] = buffered_cs_lots['geometry'].buffer(50)

# Calculate the total adjacent traffic volume for each parking lot
def calculate_total_adjacent_traffic(parking_lot, road_traffic_grouped):
    total_traffic = 0
    for road in road_traffic_grouped.itertuples():
        if parking_lot.intersects(road.geometry):
            total_traffic += road.total_count
    return total_traffic

# Add a new column to cs_lots for total adjacent traffic
cs_lots_traffic['traffic'] = buffered_cs_lots.apply(
    lambda row: calculate_total_adjacent_traffic(row['geometry'], road_traffic_projected), axis=1
).astype(int)  # Calculate total adjacent traffic using roads.

# Re-project back to the original CRS
cs_lots_traffic = cs_lots_traffic.to_crs('EPSG:4326')

# Print the results
cs_lots_traffic

In [ ]:
cs_lots_traffic

In [ ]:
cs_lots_traffic['traffic'].mean()

In [ ]:
pop_gdf = gpd.read_file("data/population.geojson")
pop_gdf.set_crs("EPSG:3857", allow_override=True, inplace=True)

income_gdf = gpd.read_file("data/income.geojson")
income_gdf.set_crs("EPSG:3857", allow_override=True, inplace=True)


In [ ]:
pop_gdf = pop_gdf.to_crs("EPSG:4326")
income_gdf = income_gdf.to_crs("EPSG:4326")


In [ ]:
lots_proj = lots.to_crs("EPSG:3857")

lots_proj["geometry"] = lots_proj.geometry.buffer(50)


lots_buffer = lots_proj.to_crs("EPSG:4326")


print("lots_buffer bounds:", lots_buffer.total_bounds)

In [ ]:

lots_pop = gpd.sjoin(lots_buffer, pop_gdf[['Population', 'geometry']], how='left', predicate='intersects')

lots_pop = lots_pop.drop(columns=['index_right'], errors='ignore')

lots_pop_income = gpd.sjoin(lots_pop, income_gdf[['Median Income', 'geometry']], how='left', predicate='intersects')
lots_pop_income = lots_pop_income.drop(columns=['index_right'], errors='ignore')


In [ ]:
lots_pop_income['Median Income'].isna().sum()

In [ ]:
lots_pop_income['Median Income'].fillna((lots_pop_income['Median Income'].mean()), inplace=True)

In [ ]:
lots_pop_income_agg = lots_pop_income.groupby('osmid').agg({
    'geometry': 'first',  
    'amenity': 'first',  
    'access': 'first',
    'fee': 'first',
    'parking': 'first',
    'capacity': 'first',
    'zone_name': 'first',
    'zone_type': 'first',
    'Population': 'mean',       
    'Median Income': 'mean'   
}).reset_index()

In [ ]:
lots_pop_income_agg['Population'].isna().sum()

In [ ]:
merged_features = cs_lots_traffic.merge(lots_pop_income_agg[['osmid', 'Population', 'Median Income']], on='osmid', how='left')

In [ ]:
zoning_suitability = {
    'Commercial': 1.0, 
    'Office': 0.8, 
    'Mixed Use': 0.7, 
    'Transit': 0.6,
    'Industrial': 0.5, 
    'Planned': 0.5, 
    'Other': 0.4, 
    'Residential High': 0.3,
    'Residential Medium': 0.2, 
    'Residential Low': 0.1, 
    'Agricultural': 0.0,
    'Open Space': 0.0,
    'Multiple': 0.7 
}

In [ ]:
merged_features['zoning_score'] = merged_features['zone_type'].map(zoning_suitability) * 0.1

In [ ]:
merged_features

In [ ]:
# import geopandas as gpd
# from shapely.geometry import Point

# # Step 1: Load your datasets (adjust names as per your code)
# # Assuming cs_gdf is the chargers GeoDataFrame and cs_lots_scored is the parking lots GeoDataFrame
# chargers = cs_gdf  # Chargers GeoDataFrame
# parking_lots = cs_lots_scored  # Parking lots GeoDataFrame

# # Step 2: Ensure both GeoDataFrames use the same coordinate reference system (CRS)
# # Start with WGS84 (EPSG:4326), then project to a meter-based CRS for accurate distances
# chargers = chargers.to_crs(epsg=4326)
# parking_lots = parking_lots.to_crs(epsg=4326)

# # Project to UTM zone 11N (EPSG:32611) for San Diego, assuming that's your area
# chargers_proj = chargers.to_crs(epsg=32611)
# parking_lots_proj = parking_lots.to_crs(epsg=32611)

# # Step 3: Buffer point parking lots to create small polygons (e.g., 10 meters radius)
# def buffer_if_point(geom, buffer_distance=10):
#     if geom.geom_type == 'Point':
#         return geom.buffer(buffer_distance)
#     else:
#         return geom

# parking_lots_proj['geometry'] = parking_lots_proj['geometry'].apply(
#     lambda g: buffer_if_point(g, buffer_distance=10)
# )

# # Step 4: Perform a spatial join to find chargers within each parking lot
# chargers_in_lots = gpd.sjoin(
#     chargers_proj,
#     parking_lots_proj,
#     how='inner',
#     predicate='within'
# )

# # Step 5: Count chargers per parking lot
# # Group by parking lot identifier ('osmid' from parking_lots_proj)
# charger_counts = chargers_in_lots.groupby('osmid').size().reset_index(name='charger_count')

# # Step 6: Merge charger counts back into the parking lots GeoDataFrame
# parking_lots_proj = parking_lots_proj.merge(
#     charger_counts,
#     on='osmid',
#     how='left'
# )

# # Step 7: Fill missing values (no chargers) with 0
# parking_lots_proj['charger_count'] = parking_lots_proj['charger_count'].fillna(0).astype(int)

# # Step 8: Create the binary target variable
# parking_lots_proj['has_charger'] = (parking_lots_proj['charger_count'] > 0).astype(int)

# # Step 9: Project back to WGS84 and update the original dataset
# parking_lots = parking_lots_proj.to_crs(epsg=4326)
# merged_features['has_charger'] = parking_lots['has_charger']

In [ ]:
merged_features

In [ ]:
# merged_features['demand_score'] = (
#     merged_features['distance_score'] +
#     merged_features['radius_score'] +
#     merged_features['cs_total_score'] +
#     merged_features['traffic_score'] +
#     merged_features['population_score'] +
#     merged_features['income_score'] +
#     merged_features['zoning_score']
# )

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Calculate percentile rank for each feature
features = ['distance_to_nearest_charger', 'chargers_in_radius', 'cs_total', 'traffic', 'Population', 'Median Income']
for feature in features:
    merged_features[f'percentile_{feature}'] = merged_features[feature].rank(pct=True)

# Manually assign weights based on expert knowledge
merged_features['distance_score'] = merged_features['percentile_distance_to_nearest_charger'] * 0.5
merged_features['radius_score'] = merged_features['percentile_chargers_in_radius'] * -0.3
merged_features['cs_total_score'] = merged_features['percentile_cs_total'] * -0.2
merged_features['traffic_score'] = merged_features['percentile_traffic'] * 0.1
merged_features['population_score'] = merged_features['percentile_Population'] * 0.2
merged_features['income_score'] = merged_features['percentile_Median Income'] * 0.1

# Calculate the combined demand score including the traffic score
merged_features['demand_score'] = (
    merged_features['distance_score'] +
    merged_features['radius_score'] +
    merged_features['cs_total_score'] +
    merged_features['traffic_score'] +
    merged_features['population_score'] +
    merged_features['income_score'] +
    merged_features['zoning_score']
)

# Normalize the demand score to a scale of 0 to 1
min_max_scaler = MinMaxScaler()
merged_features['demand_score'] = min_max_scaler.fit_transform(merged_features[['demand_score']])
print(merged_features.shape)
merged_features.head(2)

In [ ]:
merged_features.columns

In [ ]:
%%time

# Import necessary libraries
import folium
import branca.colormap as cm

# Define a colormap for the demand score
demand_score_colormap = cm.linear.YlOrRd_09.scale(merged_features['demand_score'].min(), merged_features['demand_score'].max())

# Ensure CRS is WGS84 (EPSG:4326) for compatibility with Folium
merged_features = merged_features.to_crs(epsg=4326)

# Initialize the folium map centered on San Diego
m4 = folium.Map(location=[32.7157, -117.1611], zoom_start=11, tiles="cartodbpositron")

# Add the color scale legend to the map
demand_score_colormap.caption = "Demand Score"
demand_score_colormap.add_to(m4)

# Add parking lots to the map
for _, row in merged_features.iterrows():
    color = demand_score_colormap(row['demand_score'])
    
    # Create detailed tooltip information
    tooltip_text = (
        f"Parking Lot ID: {row.osmid}<br>"
        f"Demand Score: {row.demand_score:.2f}<br>"
        f"Zone Name: {row.zone_name}<br>"
        f"Zone Type: {row.zone_type}<br>"
        f"ZIP Code: {row.zip}<br>"
        f"------Geographic Features------<br>"
        f"Distance to Nearest Charger: {row.distance_to_nearest_charger:.2f} meters<br>"
        f"Chargers in Radius: {row.chargers_in_radius}<br>"
        f"Charging Points Total: {row.cs_total}<br>"
        f"Traffic Volume: {row.traffic}<br>"
        f"Population: {row.Population}<br>"
        f"Median Income: {row['Median Income']}<br>"
        f"------Percentile Scores------<br>"
        f"Distance Percentile: {row.percentile_distance_to_nearest_charger:.2f}<br>"
        f"Radius Chargers Percentile: {row.percentile_chargers_in_radius:.2f}<br>"
        f"Total Chargers Percentile: {row.percentile_cs_total:.2f}<br>"
        f"Traffic Percentile: {row.percentile_traffic:.2f}<br>"
        f"Population Percentile: {row.percentile_Population:.2f}<br>"
        f"Income Percentile: {row['percentile_Median Income']:.2f}<br>"
        f"------Score Components------<br>"
        f"Distance Score: {row.distance_score:.2f}<br>"
        f"Radius Score: {row.radius_score:.2f}<br>"
        f"Total Charging Points Score: {row.cs_total_score:.2f}<br>"
        f"Traffic Score: {row.traffic_score:.2f}<br>"
        f"Population Score: {row.population_score:.2f}<br>"
        f"Income Score: {row.income_score:.2f}<br>"
        f"Zoning Score: {row.zoning_score:.2f}"
    )

    if isinstance(row.geometry, Point):
        # Add a small circle for point geometries
        folium.Circle(
            location=(row.geometry.y, row.geometry.x),
            radius=20,  # Small circle radius
            color=color,
            fill=True,
            fill_opacity=0.7,
            tooltip=folium.Tooltip(tooltip_text),
        ).add_to(m4)
    else:
        # Add the parking lot geometry to the map
        folium.GeoJson(
            row.geometry,
            style_function=lambda feature, color=color: {
                'fillColor': color,
                'color': color,
                'weight': 1,
                'fillOpacity': 0.7,
                'opacity': 0.7
            },
            tooltip=folium.Tooltip(tooltip_text),
        ).add_to(m4)

# Add legend explanation
legend_html = '''
<div style="position: fixed; 
     bottom: 50px; left: 50px; width: 250px; height: auto;
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color: white; padding: 10px; border-radius: 5px;">
     <b>Demand Score Explanation</b><br>
     Darker colors indicate higher demand for new charging stations<br>
     Score factors considered:<br>
     - Distance to nearest charger (weight: 0.5)<br>
     - Chargers in radius (weight: -0.3)<br>
     - Existing charging points (weight: -0.2)<br>
     - Traffic volume (weight: 0.1)<br>
     - Population density (weight: 0.2)<br>
     - Income level (weight: 0.1)<br>
</div>
'''
m4.get_root().html.add_child(folium.Element(legend_html))

# Add title at the top
title_html = '''
<h3 align="center" style="font-size:20px; font-family: 'Arial'; margin-top: 10px;">
    <b>Parking Lot Charging Station Demand Score Map</b>
</h3>
'''
m4.get_root().html.add_child(folium.Element(title_html))

# Display the map
print("Parking Lot Demand Score Map - Darker colors indicate higher demand for new charging stations")
m4

In [ ]:
%%time

# Define a colormap for the demand score
demand_score_colormap = cm.linear.YlOrRd_09.scale(cs_lots_scored['demand_score'].min(), cs_lots_scored['demand_score'].max())

# Ensure CRS is WGS84 (EPSG:4326) for compatibility with Folium
cs_lots_scored = merged_features.to_crs(epsg=4326)

# Initialize the folium map centered on San Diego
m4 = folium.Map(location=[32.7157, -117.1611], zoom_start=11, tiles="cartodbpositron")

# Add the color scale legend to the map
demand_score_colormap.caption = "Demand Score"
demand_score_colormap.add_to(m4)

# Add parking lots to the map
for _, row in cs_lots_scored.iterrows():
    color = demand_score_colormap(row['demand_score'])
    
    tooltip_text = (
        f"Parking Lot OSMID: {row.osmid}<br>"
        f"Demand Score: {row.demand_score:.2f}<br>"
        f"Zone Name: {row.zone_name}<br>"
        f"Distance to Nearest Charger: {row.distance_to_nearest_charger:.2f} meters<br>"
        f"Chargers in Radius: {row.chargers_in_radius}<br>"
        f"CS Total: {row.cs_total}<br>"
        f"Traffic: {row.traffic}<br>"
        f"Percentile Distance Score: {row.percentile_distance_to_nearest_charger:.2f}<br>"
        f"Percentile Radius Score: {row.percentile_chargers_in_radius:.2f}<br>"
        f"Percentile CS Total Score: {row.percentile_cs_total:.2f}<br>"
        f"Distance Score: {row.distance_score:.2f}<br>"
        f"Radius Score: {row.radius_score:.2f}<br>"
        f"CS Total Score: {row.cs_total_score:.2f}"
    )

    if isinstance(row.geometry, Point):
        # Add a small circle for point geometries
        folium.Circle(
            location=(row.geometry.y, row.geometry.x),
            radius=20,  # Small circle radius
            color=color,
            fill=True,
            fill_opacity=0.6,
            tooltip=folium.Tooltip(tooltip_text),
        ).add_to(m4)
    else:
        # Add the parking lot geometry to the map
        folium.GeoJson(
            row.geometry,
            style_function=lambda feature, color=color: {
                'fillColor': color,
                'color': color,
                'weight': 1,
                'fillOpacity': 0.6,
                'opacity': 0.6
            },
            tooltip=folium.Tooltip(tooltip_text),
        ).add_to(m4)

# Display the map
print("Map of parking lots with color-coded demand scores")
m4


In [ ]:
merged_features.groupby('zip').size()

In [ ]:
%%time
import folium
import branca.colormap as cm
from folium.plugins import Draw, MousePosition
import pandas as pd
import json

# Define a colormap for the demand score
demand_score_colormap = cm.linear.YlOrRd_09.scale(merged_features['demand_score'].min(), merged_features['demand_score'].max())

# Ensure CRS is WGS84 (EPSG:4326) for compatibility with Folium
merged_features = merged_features.to_crs(epsg=4326)

# Initialize the folium map centered on San Diego
m4 = folium.Map(location=[32.7157, -117.1611], zoom_start=11, tiles="cartodbpositron")

# Add the color scale legend to the map
demand_score_colormap.caption = "Demand Score"
demand_score_colormap.add_to(m4)

# Add mouse position to show coordinates
MousePosition().add_to(m4)

# Add drawing tools
Draw(export=True).add_to(m4)

# Create feature groups for regular and highlighted parking lots
regular_parking_fg = folium.FeatureGroup(name="All Parking Lots")
highlighted_parking_fg = folium.FeatureGroup(name="Recommended Parking Lots")

# Add parking lots to the regular feature group
for _, row in merged_features.iterrows():
    color = demand_score_colormap(row['demand_score'])
    
    # Create detailed tooltip information
    tooltip_text = (
        f"Parking Lot ID: {row.osmid}<br>"
        f"Demand Score: {row.demand_score:.2f}<br>"
        f"Zone Name: {row.zone_name}<br>"
        f"Zone Type: {row.zone_type}<br>"
        f"ZIP Code: {row.zip}<br>"
        f"------Geographic Features------<br>"
        f"Distance to Nearest Charger: {row.distance_to_nearest_charger:.2f} meters<br>"
        f"Chargers in Radius: {row.chargers_in_radius}<br>"
        f"Charging Points Total: {row.cs_total}<br>"
        f"Traffic Volume: {row.traffic}<br>"
        f"Population: {row.Population}<br>"
        f"Median Income: {row['Median Income']}<br>"
        f"------Percentile Scores------<br>"
        f"Distance Percentile: {row.percentile_distance_to_nearest_charger:.2f}<br>"
        f"Radius Chargers Percentile: {row.percentile_chargers_in_radius:.2f}<br>"
        f"Total Chargers Percentile: {row.percentile_cs_total:.2f}<br>"
        f"Traffic Percentile: {row.percentile_traffic:.2f}<br>"
        f"Population Percentile: {row.percentile_Population:.2f}<br>"
        f"Income Percentile: {row['percentile_Median Income']:.2f}<br>"
        f"------Score Components------<br>"
        f"Distance Score: {row.distance_score:.2f}<br>"
        f"Radius Score: {row.radius_score:.2f}<br>"
        f"Total Charging Points Score: {row.cs_total_score:.2f}<br>"
        f"Traffic Score: {row.traffic_score:.2f}<br>"
        f"Population Score: {row.population_score:.2f}<br>"
        f"Income Score: {row.income_score:.2f}<br>"
        f"Zoning Score: {row.zoning_score:.2f}"
    )

    if isinstance(row.geometry, Point):
        # Add a small circle for point geometries
        folium.Circle(
            location=(row.geometry.y, row.geometry.x),
            radius=20,  # Small circle radius
            color=color,
            fill=True,
            fill_opacity=0.7,
            tooltip=folium.Tooltip(tooltip_text),
        ).add_to(regular_parking_fg)
    else:
        # Add the parking lot geometry to the map
        folium.GeoJson(
            row.geometry,
            style_function=lambda feature, color=color: {
                'fillColor': color,
                'color': color,
                'weight': 1,
                'fillOpacity': 0.7,
                'opacity': 0.7
            },
            tooltip=folium.Tooltip(tooltip_text),
        ).add_to(regular_parking_fg)

# Add the regular feature group to the map
regular_parking_fg.add_to(m4)

# Add layer control to toggle feature groups
folium.LayerControl().add_to(m4)

# Add legend explanation
legend_html = '''
<div style="position: fixed; 
     bottom: 50px; left: 50px; width: 250px; height: auto;
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color: white; padding: 10px; border-radius: 5px;">
     <b>Demand Score Explanation</b><br>
     Darker colors indicate higher demand for new charging stations<br>
     Score factors considered:<br>
     - Distance to nearest charger (weight: 0.5)<br>
     - Chargers in radius (weight: -0.3)<br>
     - Existing charging points (weight: -0.2)<br>
     - Traffic volume (weight: 0.1)<br>
     - Population density (weight: 0.2)<br>
     - Income level (weight: 0.1)<br>
</div>
'''
m4.get_root().html.add_child(folium.Element(legend_html))

# Add title at the top
title_html = '''
<h3 align="center" style="font-size:20px; font-family: 'Arial'; margin-top: 10px;">
    <b>Parking Lot Charging Station Demand Score Map</b>
</h3>
'''
m4.get_root().html.add_child(folium.Element(title_html))

# Prepare data for JavaScript filtering
# Create a simplified GeoJSON representation of the merged_features dataframe
# Convert merged_features GeoDataFrame to GeoJSON format
geojson_data = json.loads(merged_features.to_json())

# Extract just the properties we need for each feature
js_data = []
for feature in geojson_data['features']:
    props = feature['properties']
    if props:
        item = {
            'osmid': props.get('osmid'),
            'zip': props.get('zip'),
            'demand_score': props.get('demand_score'),
            'geometry': feature['geometry']
        }
        js_data.append(item)

# Create a JavaScript function to filter and highlight the recommended locations
js_filter_code = """
// Define the filter function
function filterByZipAndCount(zip, count) {
    // Clear previous recommendations
    document.getElementById('recommendations').innerHTML = '';
    
    // Get the map object
    var map = this.map;
    
    // Clear previous highlighted feature group and create a new one
    map.eachLayer(function(layer) {
        if (layer.options && layer.options.name === 'Recommended Parking Lots') {
            map.removeLayer(layer);
        }
    });
    
    var highlightedGroup = L.featureGroup().addTo(map);
    highlightedGroup.options = {name: 'Recommended Parking Lots'};
    
    // Get all data from the merged_features DataFrame
    var parkingData = %s;
    
    // Filter data by ZIP code if provided
    var filteredData = parkingData;
    if (zip && zip.trim() !== '') {
        filteredData = parkingData.filter(function(item) {
            return item.zip && item.zip.toString() === zip.trim();
        });
    }
    
    // Sort by demand score (descending)
    filteredData.sort(function(a, b) {
        return b.demand_score - a.demand_score;
    });
    
    // Limit to the requested number of results
    var numRecommendations = parseInt(count) || 5;  // Default to 5 if not specified
    var topRecommendations = filteredData.slice(0, numRecommendations);
    
    // Add recommendations to the list and map
    topRecommendations.forEach(function(item, index) {
        // Add to recommendations list
        var recItem = document.createElement('div');
        recItem.style.padding = '5px';
        recItem.style.borderBottom = '1px solid #ccc';
        recItem.innerHTML = '<b>#' + (index + 1) + ':</b> ID: ' + item.osmid + 
                          ', Score: ' + item.demand_score.toFixed(2) + 
                          ', ZIP: ' + item.zip;
        document.getElementById('recommendations').appendChild(recItem);
        
        // Add highlighted marker to the map
        var coords = item.geometry.coordinates;
        var lat, lng;
        
        if (item.geometry.type === 'Point') {
            lng = coords[0];
            lat = coords[1];
            
            L.circleMarker([lat, lng], {
                radius: 25,
                color: '#FF4500',
                weight: 3,
                opacity: 1,
                fillColor: '#FF4500',
                fillOpacity: 0.7
            }).bindTooltip(
                'Rank #' + (index + 1) + '<br>' +
                'ID: ' + item.osmid + '<br>' +
                'Demand Score: ' + item.demand_score.toFixed(2) + '<br>' +
                'ZIP: ' + item.zip
            ).addTo(highlightedGroup);
        } else {
            // For polygons and other geometries
            var geojsonFeature = {
                "type": "Feature",
                "properties": {
                    "rank": index + 1,
                    "osmid": item.osmid,
                    "demand_score": item.demand_score,
                    "zip": item.zip
                },
                "geometry": item.geometry
            };
            
            L.geoJSON(geojsonFeature, {
                style: function(feature) {
                    return {
                        color: '#FF4500',
                        weight: 3,
                        opacity: 1,
                        fillColor: '#FF4500',
                        fillOpacity: 0.7
                    };
                }
            }).bindTooltip(function(layer) {
                return 'Rank #' + (index + 1) + '<br>' +
                       'ID: ' + item.osmid + '<br>' +
                       'Demand Score: ' + item.demand_score.toFixed(2) + '<br>' +
                       'ZIP: ' + item.zip;
            }).addTo(highlightedGroup);
        }
    });
    
    // Show the recommendations panel
    document.getElementById('rec-panel').style.display = 'block';
    
    // Update summary
    if (topRecommendations.length === 0) {
        document.getElementById('rec-summary').innerHTML = 'No locations found matching the criteria.';
    } else {
        document.getElementById('rec-summary').innerHTML = 'Showing top ' + 
            topRecommendations.length + ' recommended locations' + 
            (zip && zip.trim() !== '' ? ' in ZIP ' + zip : '') + '.';
    }
}
"""

# Insert data into JavaScript function
js_filter_code = js_filter_code % json.dumps(js_data)

# Create the filter panel HTML
filter_html = """
<div style="position: fixed; 
     top: 10px; right: 10px; width: 300px; height: auto;
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color: white; padding: 10px; border-radius: 5px;">
    <h4 style="margin-top: 0;">Filter Recommendations</h4>
    <div style="margin-bottom: 10px;">
        <label for="zip-input">ZIP Code (optional):</label>
        <input type="text" id="zip-input" style="width: 100%; padding: 5px; margin-top: 5px;">
    </div>
    <div style="margin-bottom: 10px;">
        <label for="count-input">Number of Recommendations:</label>
        <input type="number" id="count-input" min="1" max="50" value="5" style="width: 100%; padding: 5px; margin-top: 5px;">
    </div>
    <button onclick="filterByZipAndCount(
        document.getElementById('zip-input').value, 
        document.getElementById('count-input').value
    )" style="width: 100%; padding: 8px; background-color: #4CAF50; color: white; border: none; border-radius: 4px; cursor: pointer;">
        Show Recommendations
    </button>
    
    <div id="rec-panel" style="display: none; margin-top: 15px; max-height: 300px; overflow-y: auto;">
        <h4 style="margin-top: 0;">Top Recommendations</h4>
        <div id="rec-summary" style="margin-bottom: 10px; font-style: italic;"></div>
        <div id="recommendations"></div>
    </div>
</div>
"""

# Add the filter panel and JavaScript to the map
m4.get_root().html.add_child(folium.Element(filter_html))
m4.get_root().script.add_child(folium.Element(f'<script>{js_filter_code}</script>'))

# Display the map
print("Interactive Parking Lot Demand Score Map with ZIP filtering and recommendation highlights")
m4.save("new.html")

In [ ]:
%%time
# Ensure CRS is WGS84 (EPSG:4326) for compatibility with Folium
cs_lots_scored = merged_features.to_crs(epsg=4326)

# Initialize the folium map centered on San Diego
m3 = folium.Map(location=[32.7157, -117.1611], zoom_start=11, tiles="cartodbpositron")

# Function to get coordinates based on geometry type
def get_coords(geometry):
    if isinstance(geometry, (Point, Polygon, MultiPolygon)):
        centroid = geometry.centroid
        return centroid.y, centroid.x
    return np.nan, np.nan

# Add intersection points to the map
for _, row in cs_lots_scored.iterrows():
    lat, lon = get_coords(row.geometry)
    if np.isnan(lat) or np.isnan(lon):
        continue  # Skip invalid coordinates

    # Add a circle marker for each intersection
    folium.CircleMarker(
        location=(lat, lon),  # Latitude and Longitude
        radius=3,
        color="red",
        fill=True,
        fill_opacity=0.5,
        popup=folium.Popup(f"Intersection OSMID: {row.osmid}<br>Zone Name: {row.zone_name}", max_width=250),
    ).add_to(m3)

    # Add buffer zones as circles
    folium.Circle(
        location=(lat, lon),
        radius=row.distance_to_nearest_charger,  # Distance in meters
        color="blue",  # Circle line color
        weight=2,  # Line thickness
        opacity=0.3,  # Line opacity
        fill=True,
        fill_opacity=0.01,  # Adjust the fill opacity separately
    ).add_to(m3)

# Display the map
print("Map of parking lots with a radius to the nearest public charger")
m3

In [ ]:
import folium
import branca.colormap as cm
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML

# Create filter controls
# Get all available ZIP codes for the dropdown menu
available_zips = sorted(merged_features['zip'].unique())
zip_dropdown = widgets.Dropdown(
    options=[('All ZIP Codes', None)] + [(str(z), z) for z in available_zips],
    value=None,
    description='ZIP Code:',
    style={'description_width': 'initial'}
)

# Create a slider for selecting the number of recommendations
num_recommendations = widgets.IntSlider(
    value=5,
    min=1,
    max=20,
    step=1,
    description='Number of Recommendations:',
    style={'description_width': 'initial'}
)

# Create a button
filter_button = widgets.Button(
    description='Show Recommended Locations',
    button_style='success', 
    icon='filter'
)

# Output area for displaying the map
output = widgets.Output()

def create_map_with_filter(zip_code=None, num_recommendations=5):
    """
    Create a map with filtering functionality
    
    Parameters:
    zip_code (str, optional): ZIP code to filter by. Default is None (no filtering).
    num_recommendations (int, optional): Number of recommended locations. Default is 5.
    
    Returns:
    folium.Map: A map with filtering and highlighted recommended locations
    """
    # Ensure CRS is WGS84 (EPSG:4326)
    features_data = merged_features.to_crs(epsg=4326)
    
    # Filter data if a ZIP code is provided
    if zip_code:
        filtered_data = features_data[features_data['zip'] == zip_code]
    else:
        filtered_data = features_data
    
    # Sort by demand score and select the top N recommendations
    top_recommendations = filtered_data.sort_values('demand_score', ascending=False).head(num_recommendations)
    
    # Create a color map
    colormap = cm.linear.YlOrRd_09.scale(features_data['demand_score'].min(), features_data['demand_score'].max())
    
    # Create the map
    m = folium.Map(location=[32.7157, -117.1611], zoom_start=11, tiles='CartoDB positron')
    
    # Add a color legend
    colormap.caption = "Demand Score"
    colormap.add_to(m)
    
    # Create a layer for displaying all parking lots (initially hidden)
    all_parking_fg = folium.FeatureGroup(name="All Parking Lots", show=False)
    
    # Add all parking lots using the color mapping
    for _, row in features_data.iterrows():
        color = colormap(row['demand_score'])
        
        # Basic tooltip
        tooltip = f"ID: {row.osmid}, Score: {row.demand_score:.2f}, ZIP: {row.zip}"
        
        if hasattr(row.geometry, 'y') and hasattr(row.geometry, 'x'):
            # Point geometry
            folium.Circle(
                location=(row.geometry.y, row.geometry.x),
                radius=15,
                color=color,
                fill=True,
                fill_opacity=0.6,
                tooltip=tooltip
            ).add_to(all_parking_fg)
        else:
            # Try adding polygons or other geometries
            try:
                folium.GeoJson(
                    row.geometry,
                    style_function=lambda x, color=color: {
                        'fillColor': color,
                        'color': color,
                        'weight': 1,
                        'fillOpacity': 0.6
                    },
                    tooltip=tooltip
                ).add_to(all_parking_fg)
            except Exception as e:
                pass
    
    # Create a layer to highlight recommended locations (default: visible)
    recommendations_fg = folium.FeatureGroup(name="Recommended Parking Lots", show=True)
    
    # Add recommended locations using a prominent orange-red color
    for i, (_, row) in enumerate(top_recommendations.iterrows()):
        # Detailed tooltip including ranking
        tooltip = (
            f"Rank #{i+1}<br>"
            f"ID: {row.osmid}<br>"
            f"Demand Score: {row.demand_score:.2f}<br>"
            f"ZIP: {row.zip}<br>"
            f"Zone Type: {row.zone_type}"
        )
        
        if hasattr(row.geometry, 'y') and hasattr(row.geometry, 'x'):
            # Point geometry
            folium.Circle(
                location=(row.geometry.y, row.geometry.x),
                radius=25,
                color='#FF4500',
                weight=3,
                fill=True,
                fill_opacity=0.7,
                tooltip=tooltip
            ).add_to(recommendations_fg)
        else:
            # Try adding polygons or other geometries
            try:
                folium.GeoJson(
                    row.geometry,
                    style_function=lambda x: {
                        'fillColor': '#FF4500',
                        'color': '#FF4500',
                        'weight': 3,
                        'fillOpacity': 0.7
                    },
                    tooltip=tooltip
                ).add_to(recommendations_fg)
            except Exception as e:
                pass
    
    # Add layers to the map
    all_parking_fg.add_to(m)
    recommendations_fg.add_to(m)
    
    # Add a layer control and keep it expanded
    folium.LayerControl(collapsed=False).add_to(m)
    
    # Add title and description
    title_html = f'''
    <div style="position: fixed; 
         top: 10px; left: 100px; width: 600px; height: auto;
         font-size:20px; font-family: 'Arial'; z-index:9999; 
         background-color: white; padding: 10px; border-radius: 5px; border:2px solid grey;
         text-align: center;">
         <b>Parking Lot Demand Score Map</b><br>
         <span style="font-size:14px;">
         {f'Showing top {len(top_recommendations)} recommendations in ZIP code {zip_code}' 
         if zip_code else f'Showing top {len(top_recommendations)} recommendations across all ZIP codes'}
         </span>
    </div>
    '''
    m.get_root().html.add_child(folium.Element(title_html))
    
    # Add a results list to the map
    results_html = f'''
    <div style="position: fixed; 
         bottom: 10px; right: 10px; width: 300px; max-height: 300px;
         font-size:12px; font-family: 'Arial'; z-index:9999; 
         background-color: white; padding: 10px; border-radius: 5px; border:2px solid grey;
         overflow-y: auto;">
         <b>Top {len(top_recommendations)} Recommendations:</b><br>
    '''
    
    for i, (_, row) in enumerate(top_recommendations.iterrows()):
        results_html += f'''
        <div style="margin: 5px 0; padding: 5px; border-bottom: 1px solid #eee;">
            <b>#{i+1}:</b> ID: {row.osmid}<br>
            Score: {row.demand_score:.2f}<br>
            ZIP: {row.zip}<br>
            Zone: {row.zone_type if 'zone_type' in row else 'N/A'}
        </div>
        '''
    
    results_html += '</div>'
    m.get_root().html.add_child(folium.Element(results_html))
    
    # Return the map
    return m

# Callback function when the button is clicked
def on_button_clicked(b):
    with output:
        output.clear_output()
        print(f"Creating map... ZIP Code: {'All' if zip_dropdown.value is None else zip_dropdown.value}, Number of Recommendations: {num_recommendations.value}")
        m = create_map_with_filter(zip_code=zip_dropdown.value, num_recommendations=num_recommendations.value)
        display(m)

# Bind the callback function to the button
filter_button.on_click(on_button_clicked)

# Layout controls
controls = widgets.VBox([
    widgets.HBox([zip_dropdown, num_recommendations]), 
    filter_button
])

# Display controls and output area
display(HTML("<h3>Charging Station Demand Score Map - Interactive Filtering</h3>"))
display(controls)
display(output)

# Load the initial map
with output:
    print("Creating initial map...")
    m = create_map_with_filter()
    display(m)


In [ ]:
import folium
from folium.plugins import MarkerCluster, MeasureControl
import branca.colormap as cm
import numpy as np
from shapely.geometry import Point, Polygon, MultiPolygon

# Create a base map centered on San Diego using the cartodbpositron tile layer
combined_map = folium.Map(location=[32.7157, -117.1611], zoom_start=12, tiles="cartodbpositron")

# Modify area color mapping with more saturated colors
color_mapping = {
    'Commercial': '#4A90E2',       # Brighter blue
    'Office': '#50C878',           # Emerald green
    'Residential High': '#E74C3C', # Bright red
    'Residential Medium': '#F4D03F', # Golden yellow
    'Residential Low': '#F9E79F',  # Light yellow
    'Residential Mixed': '#FF69B4', # Bright pink
    'Industrial': '#95A5A6',       # Silver gray
    'Mixed Use': '#2CCBCB',        # Blue-green
    'Agricultural': '#DAA520',     # Goldenrod
    'Open Space': '#87CEFA',       # Light blue
    'Planned': '#00CED1',          # Teal
    'Transit': '#9370DB',          # Medium purple
    'Other': '#5D6D7E',            # Dark gray
    'Multiple': '#9932CC'          # Dark orchid
}

# Function: Style for zoning polygons
def zoning_style(feature):
    zone_type = feature['properties']['zone_type']
    color = color_mapping.get(zone_type, 'gray')  # Default to gray if not found
    return {
        'fillColor': color, 
        'color': 'black', 
        'weight': 1,
        'fillOpacity': 0.3,
    }

# Function: Get coordinates from geometry
def get_coords(geometry):
    if isinstance(geometry, (Point, Polygon, MultiPolygon)):
        centroid = geometry.centroid
        return centroid.y, centroid.x
    return np.nan, np.nan

# Create layer groups for toggling their visibility
zoning_layer = folium.FeatureGroup(name='Zoning Areas')
parking_layer = folium.FeatureGroup(name='Public Parking Lots')
zoned_parking_layer = folium.FeatureGroup(name='Parking by Zone Type')
charger_distance_layer = folium.FeatureGroup(name='Distance to Nearest Charger')

# 1. Add zoning layer (GeoJSON file containing area polygons)
folium.GeoJson(
    zoning_data,  # GeoJSON file containing area polygons
    style_function=zoning_style,
    name='Zoning Areas'
).add_to(zoning_layer)

# 2. Add original parking data (from the first map)
for _, row in public_parking_gdf.iterrows():
    if row.geometry.geom_type == "Point":
        folium.CircleMarker(
            location=[row.geometry.y, row.geometry.x],
            radius=5,
            color="#004080",  # Dark blue border
            fill=True,
            fill_color="#0066CC",  # More saturated blue fill
            fill_opacity=0.8,  # Increase opacity
            weight=2,  # Thicker border
            tooltip="Public Parking Lot",
            popup=folium.Popup(f"Public Parking Lot<br>Type: {row.get('parking', 'Unknown')}", max_width=250),
        ).add_to(parking_layer)
    elif row.geometry.geom_type in ["Polygon", "MultiPolygon"]:
        folium.GeoJson(
            row.geometry,
            style_function=lambda x: {
                "color": "#004080",  # Dark blue border
                "fillColor": "#0066CC",  # More saturated blue fill
                "weight": 2,  # Thicker border
                "fillOpacity": 0.6,  # Increase opacity
            },
            tooltip="Non-Private Parking Lot"
        ).add_to(parking_layer)

# 3. Add parking data with zone type information (from the second map)
for _, row in lots.iterrows():
    zone_type = row['zone_type'] if 'zone_type' in row else "Unknown"
    # Get zone color with increased saturation
    zone_color = color_mapping.get(zone_type, 'gray')
    
    if row.geometry.geom_type == "Point":
        folium.CircleMarker(
            location=[row.geometry.y, row.geometry.x],
            radius=5,
            color=zone_color,  # Use the original color mapping
            fill=True,
            fill_color=zone_color,
            fill_opacity=0.9,  # Increase opacity
            weight=2,  # Increase border thickness
            tooltip=f"Zone: {zone_type}",
            popup=folium.Popup(f"Zone Type: {zone_type}<br>OSMID: {row.get('osmid', 'N/A')}", max_width=250),
        ).add_to(zoned_parking_layer)
    elif row.geometry.geom_type in ["Polygon", "MultiPolygon"]:
        folium.GeoJson(
            row.geometry,
            style_function=lambda x: {"color": zone_color, "fillColor": zone_color, "weight": 1, "fillOpacity": 0.4},
            tooltip=f"Zone: {zone_type}"
        ).add_to(zoned_parking_layer)

# 4. Add charger distance information layer (from the third map)
for _, row in merged_features.iterrows():
    lat, lon = get_coords(row.geometry)
    if np.isnan(lat) or np.isnan(lon):
        continue  # Skip invalid coordinates
    
    # Add a circular marker for each intersection with increased radius for easier clicking
    folium.CircleMarker(
        location=(lat, lon),  # Latitude and longitude
        radius=5,  # Increase radius for easier clicking
        color="red",
        fill=True,
        fill_color="red",
        fill_opacity=0.7,
        weight=2,  # Increase border thickness
        tooltip=f"OSMID: {row.osmid} (Click for details)",
        popup=folium.Popup(f"<b>Charging Station Data</b><br>OSMID: {row.osmid}<br>Zone Name: {row.zone_name}<br>Distance to Charger: {row.distance_to_nearest_charger:.1f}m", max_width=300),
    ).add_to(charger_distance_layer)
    
    # Add a circular buffer
    folium.Circle(
        location=(lat, lon),
        radius=row.distance_to_nearest_charger,  # Distance in meters
        color="blue",  # Circle line color
        weight=2,  # Line thickness
        opacity=0.3,  # Line opacity
        fill=True,
        fill_opacity=0.01,  # Adjust fill opacity separately
    ).add_to(charger_distance_layer)

# 5. Add zoning legend
opacity = 0.8
legend_html = f"""
<div style="position: fixed; 
             top: 10px; left: 10px; 
             width: 200px; height: auto; 
             background-color: rgba(255, 255, 255, {opacity}); 
             border:2px solid grey; 
             z-index: 9999; 
             font-size:14px;
             padding: 10px;">
    <b>Zoning Categories</b><br>
    <div style="line-height: 1.5;">
"""
for category, color in color_mapping.items():
    legend_html += f'<div><i style="background:{color}; width: 20px; height: 20px; display: inline-block;"></i> {category}</div>'
legend_html += "</div></div>"
combined_map.get_root().html.add_child(folium.Element(legend_html))

# 6. Add second legend for the charger distance layer
charger_legend_html = f"""
<div style="position: fixed; 
             bottom: 10px; right: 10px; 
             width: 220px; height: auto; 
             background-color: rgba(255, 255, 255, {opacity}); 
             border:2px solid grey; 
             z-index: 9999; 
             font-size:14px;
             padding: 10px;">
    <b>Charger Distance Layer</b><br>
    <div style="line-height: 1.5;">
    <div><i style="background:red; width: 10px; height: 10px; border-radius: 50%; display: inline-block;"></i> Parking Spot</div>
    <div><i style="border: 2px solid blue; width: 16px; height: 16px; border-radius: 50%; display: inline-block;"></i> Distance to Nearest Charger</div>
    </div>
</div>
"""
combined_map.get_root().html.add_child(folium.Element(charger_legend_html))

# Modify the order of adding layers to ensure the most important information is on top
# Layer order from bottom to top: 1. Zoning Areas -> 2. Public Parking Lots -> 3. Parking by Zone Type -> 4. Charger Distance

# 1. First, add the bottom layer - Zoning Areas
zoning_layer.add_to(combined_map)

# 2. Add middle layers - Public Parking Lots and Parking by Zone Type
parking_layer.add_to(combined_map)
zoned_parking_layer.add_to(combined_map)

# 3. Finally, add the top layer - Charger Distance information
charger_distance_layer.add_to(combined_map)

# 6. Create a custom layer control function to ensure proper click behavior
def create_custom_layer_control():
    custom_layer_control = '''
    <script>
    document.addEventListener("DOMContentLoaded", function() {
        // Wait for the map to finish loading
        setTimeout(function() {
            // Add additional click handling for the layer control
            var checkboxes = document.querySelectorAll('.leaflet-control-layers-overlays input[type="checkbox"]');
            
            if (checkboxes.length >= 4) {
                // By default, turn off some layers
                checkboxes[1].click(); // Turn off Public Parking Lot layer
                checkboxes[2].click(); // Turn off Zoned Parking Lot layer
                
                // Add click event handling
                checkboxes.forEach(function(checkbox, index) {
                    checkbox.addEventListener('change', function() {
                        // Record the current layer state
                        console.log("Layer " + index + " is now: " + (this.checked ? "on" : "off"));
                        
                        // If the user opens multiple conflicting layers, show a tip
                        var enabledLayers = Array.from(checkboxes).filter(cb => cb.checked).length;
                        if (enabledLayers > 2) {
                            // Create or update the tip element
                            var tipElement = document.getElementById('map-layer-tip');
                            if (!tipElement) {
                                tipElement = document.createElement('div');
                                tipElement.id = 'map-layer-tip';
                                tipElement.style.position = 'fixed';
                                tipElement.style.bottom = '40px';
                                tipElement.style.left = '50%';
                                tipElement.style.transform = 'translateX(-50%)';
                                tipElement.style.backgroundColor = 'rgba(0, 0, 0, 0.7)';
                                tipElement.style.color = 'white';
                                tipElement.style.padding = '8px 12px';
                                tipElement.style.borderRadius = '4px';
                                tipElement.style.fontSize = '14px';
                                tipElement.style.zIndex = '1000';
                                document.body.appendChild(tipElement);
                            }
                            
                            tipElement.textContent = 'Tip: Opening multiple layers at the same time may cause click conflicts. It is recommended to view layers one at a time.';
                            tipElement.style.display = 'block';
                            
                            // Hide the tip after 3 seconds
                            setTimeout(function() {
                                tipElement.style.display = 'none';
                            }, 3000);
                        }
                    });
                });
            }
        }, 1000);
    });
    </script>
    '''
    return custom_layer_control

# Add custom layer control
combined_map.get_root().html.add_child(folium.Element(create_custom_layer_control()))

# 8. Add layer control (positioned at the top right and initially collapsed to reduce clutter)
folium.LayerControl(position='topright', collapsed=True).add_to(combined_map)

# Add a small scale
MeasureControl(position='bottomleft', primary_length_unit='meters').add_to(combined_map)

# Save the map as an HTML file
combined_map.save("combined_parking_zoning_charger_map.html")

# Display the map
print("Combined Map of Parking Lots, City Zoning, and Distance to Nearest Chargers")
combined_map


In [ ]:
m4.save("final_result_map.html")

In [ ]:
# %%time

# type_cols = ['ev_dc_fast_num','ev_level1_evse_num','ev_level2_evse_num']

# cs_by_intersection = cs_gdf.groupby('osmid')[type_cols+connector_cols].sum().reset_index()
# intersection_cs = intersections.merge(cs_by_intersection, how='left', on='osmid')

# intersection_cs[type_cols] = intersection_cs[type_cols].fillna(0)
# intersection_cs[connector_cols] = intersection_cs[connector_cols].fillna(0)
# intersection_cs['cs_total'] = intersection_cs[type_cols].sum(axis=1)

# intersection_cs[connector_cols] = intersection_cs[connector_cols] > 0

# print(intersection_cs.shape)
# intersection_cs.sort_values(by='cs_total', ascending=False).head(20)

### DMV Registration Data

Now filter the charger data to get all public AFDC chargers in the SDG&E territory

In [ ]:
# # DMV
# dmv = dmv_data.copy()
# dmv = dmv[dmv['ZIP Code']!="OOS"]
# dmv['ZIP Code'] = dmv['ZIP Code'].astype(int)
# dmv = dmv[dmv['ZIP Code'].isin(sdge_zips)]
# print(dmv.shape)
# dmv.head()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (classification_report, confusion_matrix, roc_auc_score, 
                             precision_recall_curve, auc, f1_score, make_scorer,
                             precision_score, recall_score, average_precision_score)
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# For handling imbalanced data
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler, TomekLinks
from imblearn.combine import SMOTEENN, SMOTETomek
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.ensemble import BalancedRandomForestClassifier, EasyEnsembleClassifier, BalancedBaggingClassifier
from imblearn.metrics import geometric_mean_score

# Models to compare
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
try:
    from xgboost import XGBClassifier
except ImportError:
    print("XGBoost not available, skipping XGBoost models")
    XGBClassifier = None
try:
    from lightgbm import LGBMClassifier
except ImportError:
    print("LightGBM not available, skipping LightGBM models")
    LGBMClassifier = None

import warnings
warnings.filterwarnings('ignore')

# Load the dataset
df = pd.read_csv('features.csv')
print(f"Dataset shape: {df.shape}")

# Check target distribution
print(f"Target distribution:\n{df['has_charger'].value_counts()}")
print(f"Percentage of chargers: {df['has_charger'].mean() * 100:.2f}%")

# Remove geometry column as it's not usable for ML
if 'geometry' in df.columns:
    df = df.drop('geometry', axis=1)

# Check for missing values
missing_values = df.isnull().sum()
print("\nFeatures with missing values:")
print(missing_values[missing_values > 0])

# Separate categorical and numerical features
categorical_cols = ['amenity', 'access', 'fee', 'parking', 'zone_name', 'zone_type']
numerical_cols = [col for col in df.columns if col not in categorical_cols + ['has_charger', 'osmid', 'geometry']]

# Prepare data for modeling
X = df.drop(['has_charger', 'osmid'] + [col for col in categorical_cols if col in df.columns], axis=1)
y = df['has_charger']

# Feature selection based on our previous correlation analysis
important_features = ['percentile_chargers_in_radius', 'percentile_distance_to_nearest_charger', 
                     'distance_to_nearest_charger', 'percentile_cs_total', 'traffic',
                     'cs_total', 'Population', 'Median Income', 'zoning_score', 
                     'chargers_in_radius', 'demand_score', 'income_score', 'population_score']

# Select only important features
X = X[important_features]

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f"\nTraining set shape: {X_train.shape}")
print(f"Testing set shape: {X_test.shape}")
print(f"Training class distribution: {pd.Series(y_train).value_counts()}")
print(f"Testing class distribution: {pd.Series(y_test).value_counts()}")

# -----------------------------------------------------------------------------
# Feature Importance Visualization
# -----------------------------------------------------------------------------

# Calculate feature correlation with target
correlations = {}
for col in X.columns:
    correlations[col] = np.corrcoef(X[col].values, y.values)[0, 1]

correlation_series = pd.Series(correlations).abs().sort_values(ascending=False)
plt.figure(figsize=(10, 8))
correlation_series.plot(kind='bar')
plt.title('Feature Correlation with Target')
plt.tight_layout()
plt.savefig('feature_correlation.png')
plt.close()

print("\nFeature correlation with target:")
print(correlation_series)

# -----------------------------------------------------------------------------
# Imbalanced Data Handling and Model Evaluation Functions
# -----------------------------------------------------------------------------

def evaluate_model(y_true, y_pred, y_prob=None):
    """Evaluate model performance for imbalanced classification."""
    report = classification_report(y_true, y_pred)
    
    # Calculate confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    
    # Calculate precision, recall, F1 score for positive class
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    
    # Calculate ROC AUC if probabilities are provided
    roc_auc = None
    pr_auc = None
    if y_prob is not None:
        roc_auc = roc_auc_score(y_true, y_prob)
        precision_curve, recall_curve, _ = precision_recall_curve(y_true, y_prob)
        pr_auc = auc(recall_curve, precision_curve)
    
    # Calculate G-mean
    g_mean = geometric_mean_score(y_true, y_pred)
    
    results = {
        'confusion_matrix': cm,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'g_mean': g_mean,
        'roc_auc': roc_auc,
        'pr_auc': pr_auc
    }
    
    return report, results

def plot_confusion_matrix(cm, title='Confusion Matrix', filename='confusion_matrix.png'):
    """Plot confusion matrix."""
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.title(title)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig(filename)
    plt.close()

def plot_roc_curve(y_true, y_prob, title='ROC Curve', filename='roc_curve.png'):
    """Plot ROC curve."""
    from sklearn.metrics import roc_curve
    fpr, tpr, _ = roc_curve(y_true, y_prob)
    roc_auc = roc_auc_score(y_true, y_prob)
    
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.savefig(filename)
    plt.close()

def plot_precision_recall_curve(y_true, y_prob, title='Precision-Recall Curve', filename='pr_curve.png'):
    """Plot Precision-Recall curve."""
    precision, recall, _ = precision_recall_curve(y_true, y_prob)
    pr_auc = auc(recall, precision)
    
    plt.figure(figsize=(8, 6))
    plt.plot(recall, precision, color='darkorange', lw=2, label=f'PR curve (area = {pr_auc:.2f})')
    plt.axhline(y=sum(y_true)/len(y_true), color='navy', linestyle='--', label='No Skill')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title(title)
    plt.legend(loc="upper right")
    plt.savefig(filename)
    plt.close()

# -----------------------------------------------------------------------------
# First set of models: Baseline models without handling class imbalance
# -----------------------------------------------------------------------------

def run_baseline_models():
    """Run baseline models without handling class imbalance."""
    print("\n\n------ BASELINE MODELS (WITHOUT IMBALANCE HANDLING) ------\n")
    
    models = {
        'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
        'Random Forest': RandomForestClassifier(random_state=42),
        'Gradient Boosting': GradientBoostingClassifier(random_state=42),
        'SVM': SVC(probability=True, random_state=42),
        'KNN': KNeighborsClassifier(),
        'Decision Tree': DecisionTreeClassifier(random_state=42),
        'Naive Bayes': GaussianNB()
    }
    
    if XGBClassifier:
        models['XGBoost'] = XGBClassifier(random_state=42)
    
    if LGBMClassifier:
        models['LightGBM'] = LGBMClassifier(random_state=42)
    
    results = {}
    
    for name, model in models.items():
        print(f"\nTraining {name}...")
        # Create a pipeline with preprocessing
        pipeline = Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler()),
            ('model', model)
        ])
        
        # Train the model
        pipeline.fit(X_train, y_train)
        
        # Make predictions
        y_pred = pipeline.predict(X_test)
        y_prob = pipeline.predict_proba(X_test)[:, 1]
        
        # Evaluate the model
        report, metrics = evaluate_model(y_test, y_pred, y_prob)
        
        print(f"\n{name} Results:")
        print(report)
        print(f"ROC AUC: {metrics['roc_auc']:.4f}")
        print(f"PR AUC: {metrics['pr_auc']:.4f}")
        print(f"G-mean: {metrics['g_mean']:.4f}")
        
        results[name] = metrics
        
        # Plot confusion matrix
        plot_confusion_matrix(metrics['confusion_matrix'], 
                             title=f'{name} Confusion Matrix',
                             filename=f'{name.lower().replace(" ", "_")}_cm.png')
        
        # Plot ROC curve
        plot_roc_curve(y_test, y_prob, 
                      title=f'{name} ROC Curve', 
                      filename=f'{name.lower().replace(" ", "_")}_roc.png')
        
        # Plot Precision-Recall curve
        plot_precision_recall_curve(y_test, y_prob,
                                  title=f'{name} Precision-Recall Curve',
                                  filename=f'{name.lower().replace(" ", "_")}_pr.png')
    
    return results

# -----------------------------------------------------------------------------
# Second set of models: Models with resampling methods for imbalanced data
# -----------------------------------------------------------------------------

def run_resampling_models():
    """Run models with resampling methods for imbalanced data."""
    print("\n\n------ MODELS WITH RESAMPLING METHODS ------\n")
    
    base_model = LogisticRegression(max_iter=1000, random_state=42)
    
    resampling_methods = {
        'SMOTE': SMOTE(random_state=42),
        'ADASYN': ADASYN(random_state=42),
        'RandomUnderSampler': RandomUnderSampler(random_state=42),
        'TomekLinks': TomekLinks(),
        'SMOTE + ENN': SMOTEENN(random_state=42),
        'SMOTE + Tomek': SMOTETomek(random_state=42)
    }
    
    results = {}
    
    for name, resampler in resampling_methods.items():
        print(f"\nTraining with {name}...")
        
        # Create a pipeline with preprocessing and resampling
        pipeline = ImbPipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler()),
            ('resampler', resampler),
            ('model', base_model)
        ])
        
        # Train the model
        pipeline.fit(X_train, y_train)
        
        # Make predictions
        y_pred = pipeline.predict(X_test)
        y_prob = pipeline.predict_proba(X_test)[:, 1]
        
        # Evaluate the model
        report, metrics = evaluate_model(y_test, y_pred, y_prob)
        
        print(f"\nLogistic Regression with {name} Results:")
        print(report)
        print(f"ROC AUC: {metrics['roc_auc']:.4f}")
        print(f"PR AUC: {metrics['pr_auc']:.4f}")
        print(f"G-mean: {metrics['g_mean']:.4f}")
        
        results[name] = metrics
        
        # Plot confusion matrix
        plot_confusion_matrix(metrics['confusion_matrix'], 
                             title=f'LR with {name} Confusion Matrix',
                             filename=f'lr_{name.lower().replace(" ", "_").replace("+", "plus")}_cm.png')
        
        # Plot ROC curve
        plot_roc_curve(y_test, y_prob, 
                      title=f'LR with {name} ROC Curve', 
                      filename=f'lr_{name.lower().replace(" ", "_").replace("+", "plus")}_roc.png')
        
        # Plot Precision-Recall curve
        plot_precision_recall_curve(y_test, y_prob,
                                  title=f'LR with {name} Precision-Recall Curve',
                                  filename=f'lr_{name.lower().replace(" ", "_").replace("+", "plus")}_pr.png')
    
    return results

# -----------------------------------------------------------------------------
# Third set of models: Specialized algorithms for imbalanced data
# -----------------------------------------------------------------------------

def run_specialized_models():
    """Run specialized algorithms for imbalanced data."""
    print("\n\n------ SPECIALIZED ALGORITHMS FOR IMBALANCED DATA ------\n")
    
    models = {
        'BalancedRandomForest': BalancedRandomForestClassifier(random_state=42),
        'EasyEnsemble': EasyEnsembleClassifier(random_state=42),
        'BalancedBagging': BalancedBaggingClassifier(random_state=42)
    }
    
    results = {}
    
    for name, model in models.items():
        print(f"\nTraining {name}...")
        
        # Create a pipeline with preprocessing
        pipeline = Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler()),
            ('model', model)
        ])
        
        # Train the model
        pipeline.fit(X_train, y_train)
        
        # Make predictions
        y_pred = pipeline.predict(X_test)
        y_prob = pipeline.predict_proba(X_test)[:, 1]
        
        # Evaluate the model
        report, metrics = evaluate_model(y_test, y_pred, y_prob)
        
        print(f"\n{name} Results:")
        print(report)
        print(f"ROC AUC: {metrics['roc_auc']:.4f}")
        print(f"PR AUC: {metrics['pr_auc']:.4f}")
        print(f"G-mean: {metrics['g_mean']:.4f}")
        
        results[name] = metrics
        
        # Plot confusion matrix
        plot_confusion_matrix(metrics['confusion_matrix'], 
                             title=f'{name} Confusion Matrix',
                             filename=f'{name.lower()}_cm.png')
        
        # Plot ROC curve
        plot_roc_curve(y_test, y_prob, 
                      title=f'{name} ROC Curve', 
                      filename=f'{name.lower()}_roc.png')
        
        # Plot Precision-Recall curve
        plot_precision_recall_curve(y_test, y_prob,
                                  title=f'{name} Precision-Recall Curve',
                                  filename=f'{name.lower()}_pr.png')
    
    return results

# -----------------------------------------------------------------------------
# Fourth set: Cost-sensitive learning and threshold tuning
# -----------------------------------------------------------------------------

def run_cost_sensitive_learning():
    """Run cost-sensitive learning approaches."""
    print("\n\n------ COST-SENSITIVE LEARNING ------\n")
    
    # Define different class weights
    class_weights = {
        'balanced': 'balanced',
        'custom_1': {0: 1, 1: 10},
        'custom_2': {0: 1, 1: 20},
        'custom_3': {0: 1, 1: 50}
    }
    
    results = {}
    
    for weight_name, class_weight in class_weights.items():
        print(f"\nTraining with class weight: {weight_name}...")
        
        # Create a model with specified class weights
        model = LogisticRegression(class_weight=class_weight, max_iter=1000, random_state=42)
        
        # Create a pipeline with preprocessing
        pipeline = Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler()),
            ('model', model)
        ])
        
        # Train the model
        pipeline.fit(X_train, y_train)
        
        # Make predictions
        y_pred = pipeline.predict(X_test)
        y_prob = pipeline.predict_proba(X_test)[:, 1]
        
        # Evaluate the model
        report, metrics = evaluate_model(y_test, y_pred, y_prob)
        
        print(f"\nLogistic Regression with {weight_name} class weight Results:")
        print(report)
        print(f"ROC AUC: {metrics['roc_auc']:.4f}")
        print(f"PR AUC: {metrics['pr_auc']:.4f}")
        print(f"G-mean: {metrics['g_mean']:.4f}")
        
        results[weight_name] = metrics
        
        # Plot confusion matrix
        plot_confusion_matrix(metrics['confusion_matrix'], 
                             title=f'LR with {weight_name} class weight Confusion Matrix',
                             filename=f'lr_{weight_name}_cm.png')
        
    return results

def threshold_tuning():
    """Perform threshold tuning for improved performance."""
    print("\n\n------ THRESHOLD TUNING ------\n")
    
    # Use the best model from previous experiments (can be adjusted based on results)
    model = LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42)
    
    # Create a pipeline with preprocessing
    pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler()),
        ('model', model)
    ])
    
    # Train the model
    pipeline.fit(X_train, y_train)
    
    # Get probability predictions
    y_prob = pipeline.predict_proba(X_test)[:, 1]
    
    # Test different thresholds
    thresholds = np.arange(0.1, 0.9, 0.1)
    results = {}
    
    for threshold in thresholds:
        # Apply threshold
        y_pred = (y_prob >= threshold).astype(int)
        
        # Evaluate the model
        report, metrics = evaluate_model(y_test, y_pred, y_prob)
        
        print(f"\nThreshold = {threshold:.1f} Results:")
        print(f"Precision: {metrics['precision']:.4f}")
        print(f"Recall: {metrics['recall']:.4f}")
        print(f"F1-score: {metrics['f1_score']:.4f}")
        print(f"G-mean: {metrics['g_mean']:.4f}")
        
        results[threshold] = metrics
    
    # Visualize threshold impact
    thresholds_list = list(results.keys())
    precision_list = [results[t]['precision'] for t in thresholds_list]
    recall_list = [results[t]['recall'] for t in thresholds_list]
    f1_list = [results[t]['f1_score'] for t in thresholds_list]
    g_mean_list = [results[t]['g_mean'] for t in thresholds_list]
    
    plt.figure(figsize=(12, 8))
    plt.plot(thresholds_list, precision_list, 'bo-', label='Precision')
    plt.plot(thresholds_list, recall_list, 'ro-', label='Recall')
    plt.plot(thresholds_list, f1_list, 'go-', label='F1-score')
    plt.plot(thresholds_list, g_mean_list, 'mo-', label='G-mean')
    plt.xlabel('Threshold')
    plt.ylabel('Score')
    plt.title('Impact of Threshold on Classification Metrics')
    plt.grid(True)
    plt.legend()
    plt.savefig('threshold_tuning.png')
    plt.close()
    
    return results

# -----------------------------------------------------------------------------
# Fifth set: Hyperparameter tuning for best models
# -----------------------------------------------------------------------------

def hyperparameter_tuning():
    """Perform hyperparameter tuning for the best models."""
    print("\n\n------ HYPERPARAMETER TUNING ------\n")
    
    # Choose the best resampling method based on previous results (adjust as needed)
    resampler = SMOTE(random_state=42)
    
    # Models to tune
    models = {
        'Logistic Regression': {
            'model': LogisticRegression(random_state=42, max_iter=1000),
            'params': {
                'model__C': [0.001, 0.01, 0.1, 1, 10, 100],
                'model__penalty': ['l1', 'l2'],
                'model__solver': ['liblinear', 'saga'],
                'model__class_weight': ['balanced', {0: 1, 1: 10}, {0: 1, 1: 20}]
            }
        },
        'Random Forest': {
            'model': RandomForestClassifier(random_state=42),
            'params': {
                'model__n_estimators': [50, 100, 200],
                'model__max_depth': [None, 5, 10, 20],
                'model__min_samples_split': [2, 5, 10],
                'model__class_weight': ['balanced', 'balanced_subsample', None]
            }
        }
    }
    
    if XGBClassifier:
        models['XGBoost'] = {
            'model': XGBClassifier(random_state=42),
            'params': {
                'model__n_estimators': [50, 100, 200],
                'model__max_depth': [3, 5, 7],
                'model__learning_rate': [0.01, 0.1, 0.2],
                'model__scale_pos_weight': [1, 5, 10, 20]
            }
        }
    
    results = {}
    
    for name, config in models.items():
        print(f"\nTuning {name}...")
        
        # Create a pipeline with preprocessing and resampling
        pipeline = ImbPipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler()),
            ('resampler', resampler),
            ('model', config['model'])
        ])
        
        # Define scoring metrics
        scoring = {
            'precision': 'precision',
            'recall': 'recall',
            'f1': 'f1',
            'roc_auc': 'roc_auc'
        }
        
        # Define cross-validation strategy
        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        
        # Create grid search
        grid_search = GridSearchCV(
            pipeline,
            config['params'],
            cv=cv,
            scoring=scoring,
            refit='f1',  # Optimize for F1 score
            verbose=1,
            n_jobs=-1  # Use all available cores
        )
        
        # Fit grid search
        grid_search.fit(X_train, y_train)
        
        # Get best model
        best_model = grid_search.best_estimator_
        
        # Make predictions
        y_pred = best_model.predict(X_test)
        y_prob = best_model.predict_proba(X_test)[:, 1]
        
        # Evaluate the model
        report, metrics = evaluate_model(y_test, y_pred, y_prob)
        
        print(f"\n{name} Best Parameters:")
        print(grid_search.best_params_)
        print(f"\n{name} Results:")
        print(report)
        print(f"ROC AUC: {metrics['roc_auc']:.4f}")
        print(f"PR AUC: {metrics['pr_auc']:.4f}")
        print(f"G-mean: {metrics['g_mean']:.4f}")
        
        results[name] = {
            'best_params': grid_search.best_params_,
            'metrics': metrics
        }
        
        # Plot confusion matrix
        plot_confusion_matrix(metrics['confusion_matrix'], 
                             title=f'{name} (Tuned) Confusion Matrix',
                             filename=f'{name.lower().replace(" ", "_")}_tuned_cm.png')
        
        # Plot ROC curve
        plot_roc_curve(y_test, y_prob, 
                      title=f'{name} (Tuned) ROC Curve', 
                      filename=f'{name.lower().replace(" ", "_")}_tuned_roc.png')
        
        # Plot Precision-Recall curve
        plot_precision_recall_curve(y_test, y_prob,
                                  title=f'{name} (Tuned) Precision-Recall Curve',
                                  filename=f'{name.lower().replace(" ", "_")}_tuned_pr.png')
    
    return results

# -----------------------------------------------------------------------------
# Main execution
# -----------------------------------------------------------------------------

if __name__ == "__main__":
    # Record all results
    all_results = {}
    
    # Run baseline models
    baseline_results = run_baseline_models()
    all_results['baseline'] = baseline_results
    
    # Run models with resampling methods
    resampling_results = run_resampling_models()
    all_results['resampling'] = resampling_results
    
    # Run specialized algorithms for imbalanced data
    specialized_results = run_specialized_models()
    all_results['specialized'] = specialized_results
    
    # Run cost-sensitive learning
    cost_sensitive_results = run_cost_sensitive_learning()
    all_results['cost_sensitive'] = cost_sensitive_results
    
    # Perform threshold tuning
    threshold_results = threshold_tuning()
    all_results['threshold'] = threshold_results
    
    # Perform hyperparameter tuning for best models
    tuned_results = hyperparameter_tuning()
    all_results['tuned'] = tuned_results
    
    # -----------------------------------------------------------------------------
    # Results Comparison and Visualization
    # -----------------------------------------------------------------------------
    
    # Compare all models based on F1-score
    f1_scores = {}
    
    # Extract F1-scores from baseline models
    for model_name, metrics in baseline_results.items():
        f1_scores[f"Baseline: {model_name}"] = metrics['f1_score']
    
    # Extract F1-scores from resampling methods
    for method_name, metrics in resampling_results.items():
        f1_scores[f"Resampling: {method_name}"] = metrics['f1_score']
    
    # Extract F1-scores from specialized algorithms
    for model_name, metrics in specialized_results.items():
        f1_scores[f"Specialized: {model_name}"] = metrics['f1_score']
    
    # Extract F1-scores from cost-sensitive learning
    for weight_name, metrics in cost_sensitive_results.items():
        f1_scores[f"Cost-sensitive: {weight_name}"] = metrics['f1_score']
    
    # Extract F1-scores from tuned models
    for model_name, results in tuned_results.items():
        f1_scores[f"Tuned: {model_name}"] = results['metrics']['f1_score']
    
    # Sort models by F1-score
    sorted_f1_scores = {k: v for k, v in sorted(f1_scores.items(), key=lambda item: item[1], reverse=True)}
    
    # Visualize model comparison
    plt.figure(figsize=(14, 10))
    plt.barh(list(sorted_f1_scores.keys()), list(sorted_f1_scores.values()))
    plt.xlabel('F1-score')
    plt.title('Model Comparison (F1-score)')
    plt.tight_layout()
    plt.savefig('model_comparison_f1.png')
    plt.close()
    
    print("\n\n------ MODEL COMPARISON (F1-SCORE) ------\n")
    for model_name, f1_score in sorted_f1_scores.items():
        print(f"{model_name}: {f1_score:.4f}")
    
    # Find the best model overall
    best_model_name = max(sorted_f1_scores, key=sorted_f1_scores.get)
    print(f"\nBest model: {best_model_name} with F1-score: {sorted_f1_scores[best_model_name]:.4f}")
    
    # -----------------------------------------------------------------------------
    # Feature Importance Analysis for the Best Model
    # -----------------------------------------------------------------------------
    
    # For illustrative purposes, assume the best model is Random Forest
    # This should be adjusted based on actual results
    try:
        # Create and train a Random Forest model with SMOTE resampling
        pipeline = ImbPipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler()),
            ('resampler', SMOTE(random_state=42)),
            ('model', RandomForestClassifier(n_estimators=200, random_state=42, class_weight='balanced'))
        ])
        
        # Train the model
        pipeline.fit(X_train, y_train)
        
        # Get feature importances
        rf_model = pipeline.named_steps['model']
        feature_importances = rf_model.feature_importances_
        
        # Create Series for feature importances
        feature_importance_series = pd.Series(feature_importances, index=X.columns)
        feature_importance_series = feature_importance_series.sort_values(ascending=False)
        
        # Plot feature importances
        plt.figure(figsize=(12, 8))
        feature_importance_series.plot(kind='bar')
        plt.title('Feature Importances from Random Forest')
        plt.tight_layout()
        plt.savefig('feature_importances.png')
        plt.close()
        
        print("\nFeature Importances:")
        for feature, importance in feature_importance_series.items():
            print(f"{feature}: {importance:.4f}")
    except Exception as e:
        print(f"Error in feature importance analysis: {e}")
    
    # -----------------------------------------------------------------------------
    # Final Model Implementation and Prediction on New Data
    # -----------------------------------------------------------------------------
    
    def train_final_model():
        """Train the final model based on the best configuration."""
        print("\n\n------ FINAL MODEL ------\n")
        
        # Select the best model configuration based on results
        # This is a placeholder - replace with the actual best model from previous steps
        final_pipeline = ImbPipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler()),
            ('resampler', SMOTE(random_state=42)),
            ('model', RandomForestClassifier(
                n_estimators=200, 
                max_depth=10,
                min_samples_split=5,
                class_weight='balanced',
                random_state=42
            ))
        ])
        
        # Train on the full training set
        final_pipeline.fit(X_train, y_train)
        
        # Make predictions on the test set
        y_pred = final_pipeline.predict(X_test)
        y_prob = final_pipeline.predict_proba(X_test)[:, 1]
        
        # Evaluate the final model
        report, metrics = evaluate_model(y_test, y_pred, y_prob)
        
        print("\nFinal Model Results:")
        print(report)
        print(f"ROC AUC: {metrics['roc_auc']:.4f}")
        print(f"PR AUC: {metrics['pr_auc']:.4f}")
        print(f"G-mean: {metrics['g_mean']:.4f}")
        
        # Plot confusion matrix
        plot_confusion_matrix(metrics['confusion_matrix'], 
                             title='Final Model Confusion Matrix',
                             filename='final_model_cm.png')
        
        # Plot ROC curve
        plot_roc_curve(y_test, y_prob, 
                      title='Final Model ROC Curve', 
                      filename='final_model_roc.png')
        
        # Plot Precision-Recall curve
        plot_precision_recall_curve(y_test, y_prob,
                                  title='Final Model Precision-Recall Curve',
                                  filename='final_model_pr.png')
        
        return final_pipeline, metrics
    
    # Train the final model
    final_model, final_metrics = train_final_model()
    
    # Function to predict on new parking lots
    def predict_charging_potential(model, new_data):
        """Predict charging station potential for new parking lots."""
        # Preprocess new data (similar to training data)
        # Make sure to use the same features as in training
        if isinstance(new_data, pd.DataFrame):
            new_data = new_data[X.columns]
        
        # Make predictions
        probabilities = model.predict_proba(new_data)[:, 1]
        predictions = model.predict(new_data)
        
        return predictions, probabilities
    
    # Example of how to use the prediction function
    # (You can replace this with actual new data)
    print("\n\nExample prediction on test data:")
    sample_indices = np.random.choice(X_test.index, 5, replace=False)
    sample_data = X_test.loc[sample_indices]
    
    predictions, probabilities = predict_charging_potential(final_model, sample_data)
    
    results_df = pd.DataFrame({
        'Prediction': predictions,
        'Probability': probabilities
    }, index=sample_indices)
    
    print(results_df)
    
    # Save the final model (optional)
    import pickle
    with open('evcs_prediction_model.pkl', 'wb') as file:
        pickle.dump(final_model, file)
    
    print("\nFinal model saved as 'evcs_prediction_model.pkl'")

# -----------------------------------------------------------------------------
# Simplified API for making predictions
# -----------------------------------------------------------------------------

def load_model(model_path='evcs_prediction_model.pkl'):
    """Load the trained model."""
    with open(model_path, 'rb') as file:
        model = pickle.load(file)
    return model

def predict_ev_charger_suitability(model, features_dict):
    """
    Predict EV charger suitability for a parking lot.
    
    Parameters:
    -----------
    model : trained model
        The trained machine learning model
    features_dict : dict
        Dictionary containing feature values for a parking lot
    
    Returns:
    --------
    dict
        Dictionary containing prediction results
    """
    # Convert features dictionary to DataFrame
    features_df = pd.DataFrame([features_dict])
    
    # Make sure all required features are present
    required_features = important_features
    for feature in required_features:
        if feature not in features_df.columns:
            features_df[feature] = 0  # Default value for missing features
    
    # Keep only the features used during training
    features_df = features_df[required_features]
    
    # Make prediction
    prediction = int(model.predict(features_df)[0])
    probability = float(model.predict_proba(features_df)[0, 1])
    
    # Return results
    return {
        'prediction': prediction,
        'probability': probability,
        'suitability': 'High' if probability > 0.7 else ('Medium' if probability > 0.4 else 'Low'),
        'explanation': generate_explanation(features_dict, probability)
    }

def generate_explanation(features, probability):
    """Generate a human-readable explanation for the prediction."""
    explanation = []
    
    # Add explanation based on probability
    if probability > 0.7:
        explanation.append("This parking lot has a high potential for an EV charging station.")
    elif probability > 0.4:
        explanation.append("This parking lot has a moderate potential for an EV charging station.")
    else:
        explanation.append("This parking lot has a low potential for an EV charging station.")
    
    # Add feature-specific explanations
    if 'distance_to_nearest_charger' in features and features['distance_to_nearest_charger'] > 1000:
        explanation.append("The large distance to the nearest existing charger suggests this area may be underserved.")
    
    if 'traffic' in features and features['traffic'] > 10000:
        explanation.append("The high traffic volume in this area could lead to good utilization of a charging station.")
    
    if 'Population' in features and features['Population'] > 6000:
        explanation.append("The high population density nearby supports the need for a charging station.")
    
    if 'zone_type' in features:
        if features['zone_type'] == 'Commercial':
            explanation.append("The commercial zoning is highly favorable for a charging station.")
        elif features['zone_type'] == 'Residential High':
            explanation.append("The high-density residential zoning indicates potential for good usage.")
    
    return " ".join(explanation)

# Example usage of the API
if __name__ == "__main__":
    # This would be run after the main analysis is complete
    try:
        # Load the saved model
        model = load_model()
        
        # Example features for a new parking lot
        example_features = {
            'percentile_chargers_in_radius': 0.2,
            'percentile_distance_to_nearest_charger': 0.8,
            'distance_to_nearest_charger': 1200,
            'percentile_cs_total': 0.3,
            'traffic': 15000,
            'cs_total': 0,
            'Population': 7000,
            'Median Income': 65000,
            'zoning_score': 0.9,
            'chargers_in_radius': 0,
            'demand_score': 0.7,
            'income_score': 0.5,
            'population_score': 0.7
        }
        
        # Make prediction
        result = predict_ev_charger_suitability(model, example_features)
        
        # Print result
        print("\nPrediction API Example:")
        print(f"Prediction: {'Suitable' if result['prediction'] == 1 else 'Not Suitable'}")
        print(f"Probability: {result['probability']:.4f}")
        print(f"Suitability: {result['suitability']}")
        print(f"Explanation: {result['explanation']}")
        
    except Exception as e:
        print(f"Error in prediction API example: {e}")